# Demo del MCP de Catastro Geográfico Inteligente

Este notebook demuestra el uso del MCP de Catastro Geográfico Inteligente para consultar información catastral a partir de direcciones en lenguaje natural.

## Requisitos

- Python 3.8+
- Dependencias instaladas (ver README.md)
- API de Google Maps (opcional, se usa fallback a Nominatim si no está disponible)

In [ ]:
# Importar librerías necesarias
import os
import sys
import json
import pandas as pd
import requests
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Añadir directorio raíz al path
sys.path.append('..')

# Importar módulos del proyecto
from api.utils.geocode import GeocodingService
from api.models.catastro import CatastroService

## 1. Cargar datos de ejemplo

Primero, cargamos los datos de ejemplo y mock data para las demostraciones.

In [ ]:
# Cargar mock data
with open('mock_data.json', 'r', encoding='utf-8') as f:
    mock_data = json.load(f)

# Mostrar direcciones de ejemplo
print("Direcciones de ejemplo:")
for i, direccion in enumerate(mock_data['demo_direcciones']):
    print(f"{i+1}. {direccion['direccion']}, {direccion['ciudad']}, {direccion['pais']} - {direccion['descripcion']}")

## 2. Geocodificación de direcciones

Demostración del proceso de geocodificación de direcciones utilizando Google Maps API y Nominatim como fallback.

In [ ]:
# Inicializar servicio de geocodificación
# Usar API key de Google si está disponible en variables de entorno
google_api_key = os.environ.get("GOOGLE_MAPS_API_KEY")
geocoder = GeocodingService(google_api_key)

# Función para mostrar resultados de geocodificación
def mostrar_geocodificacion(direccion, ciudad, pais):
    direccion_completa = f"{direccion}, {ciudad}, {pais}"
    print(f"Geocodificando: {direccion_completa}")
    
    # Geocodificar con Google Maps API
    if google_api_key:
        print("\nUsando Google Maps API:")
        try:
            google_result = geocoder._geocode_google(direccion_completa)
            if google_result.get("status") == "OK":
                location_data = geocoder.extract_location_data(google_result)
                print(f"Dirección formateada: {location_data.get('formatted_address')}")
                print(f"Coordenadas: {location_data.get('coordinates')}")
            else:
                print(f"Error: {google_result.get('status')}")
        except Exception as e:
            print(f"Error: {e}")
    else:
        print("API key de Google Maps no disponible.")
    
    # Geocodificar con Nominatim (fallback)
    print("\nUsando Nominatim (fallback):")
    try:
        nominatim_result = geocoder._geocode_nominatim(direccion_completa)
        if nominatim_result.get("status") == "OK":
            location_data = geocoder.extract_location_data(nominatim_result)
            print(f"Dirección formateada: {location_data.get('formatted_address')}")
            print(f"Coordenadas: {location_data.get('coordinates')}")
        else:
            print(f"Error: {nominatim_result.get('status')}")
    except Exception as e:
        print(f"Error: {e}")

# Demostrar geocodificación con la primera dirección de ejemplo
ejemplo = mock_data['demo_direcciones'][0]
mostrar_geocodificacion(ejemplo['direccion'], ejemplo['ciudad'], ejemplo['pais'])

## 3. Consulta de información catastral

Demostración de la consulta de información catastral a partir de coordenadas o direcciones.

In [ ]:
# Inicializar servicio de catastro
data_path = "../data/bogota/TPREDIO.csv"
catastro = CatastroService(data_path, google_api_key)

# Función para mostrar información catastral
def mostrar_info_catastral(resultado):
    if resultado.get("success"):
        predio = resultado.get("predio")
        html = f"""
        <div style="background-color: #f5f5f5; padding: 15px; border-radius: 5px;">
            <h3>Información Catastral</h3>
            <table>
                <tr><td><b>CHIP:</b></td><td>{predio.get('chip')}</td></tr>
                <tr><td><b>Número Predial:</b></td><td>{predio.get('numero_predial')}</td></tr>
                <tr><td><b>Dirección:</b></td><td>{predio.get('direccion')}</td></tr>
                <tr><td><b>Barrio:</b></td><td>{predio.get('barrio')}</td></tr>
                <tr><td><b>Área Terreno:</b></td><td>{predio.get('area_terreno')} m²</td></tr>
                <tr><td><b>Área Construida:</b></td><td>{predio.get('area_construida')} m²</td></tr>
                <tr><td><b>Uso:</b></td><td>{predio.get('uso_descripcion')} (Código: {predio.get('uso_codigo')})</td></tr>
                <tr><td><b>Estrato:</b></td><td>{predio.get('estrato')}</td></tr>
                <tr><td><b>Año Construcción:</b></td><td>{predio.get('anio_construccion')}</td></tr>
                <tr><td><b>Valor Catastral:</b></td><td>${predio.get('valor_catastral'):,.2f} COP</td></tr>
            </table>
        </div>
        """
        display(HTML(html))
    else:
        print(f"Error: {resultado.get('error')}")

# Usar datos mock para demostración
mock_resultado = mock_data['mock_resultados'][0]
print(f"Consulta para: {mock_resultado['direccion']['original']}")
print(f"Coordenadas: {mock_resultado['coordenadas']['lat']}, {mock_resultado['coordenadas']['lng']}")
mostrar_info_catastral({"success": True, "predio": mock_resultado['predio']})

## 4. Puntos de interés cercanos

Demostración de la búsqueda de puntos de interés cercanos a un predio.

In [ ]:
# Función para mostrar POIs cercanos
def mostrar_pois_cercanos(pois):
    if not pois:
        print("No se encontraron puntos de interés cercanos.")
        return
    
    # Crear gráfico de barras para distancias
    tipos = [poi['tipo'] for poi in pois]
    distancias = [poi['distancia'] for poi in pois]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(tipos, distancias, color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'])
    plt.xlabel('Tipo de POI')
    plt.ylabel('Distancia (metros)')
    plt.title('Distancia a Puntos de Interés Cercanos')
    plt.xticks(rotation=45)
    
    # Añadir etiquetas de valor
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 5,
                 f'{height:.1f}m',
                 ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar tabla de POIs
    html = f"""
    <div style="background-color: #f5f5f5; padding: 15px; border-radius: 5px;">
        <h3>Puntos de Interés Cercanos</h3>
        <table>
            <tr>
                <th>Tipo</th>
                <th>Nombre</th>
                <th>Distancia</th>
                <th>Dirección</th>
            </tr>
    """
    
    for poi in pois:
        html += f"""
            <tr>
                <td>{poi['tipo']}</td>
                <td>{poi['nombre']}</td>
                <td>{poi['distancia']} m</td>
                <td>{poi['direccion']}</td>
            </tr>
        """
    
    html += """
        </table>
    </div>
    """
    
    display(HTML(html))

# Usar datos mock para demostración
mock_pois = mock_data['mock_resultados'][0]['pois_cercanos']
mostrar_pois_cercanos(mock_pois)

## 5. Flujo completo del MCP

Demostración del flujo completo del MCP para consulta catastral a partir de una dirección.

In [ ]:
# Función para simular el flujo del MCP
def simular_flujo_mcp(direccion, ciudad, pais):
    print("=== Iniciando flujo MCP de Catastro Geográfico Inteligente ===")
    print("\nMCP: Bienvenido al servicio de consulta catastral. Puedo ayudarte a obtener información detallada sobre un predio a partir de su dirección.")
    
    print(f"\nUsuario: {direccion}")
    print(f"\nMCP: ¿En qué ciudad está ubicada esa dirección?")
    
    print(f"\nUsuario: {ciudad}")
    print(f"\nMCP: ¿En qué país está ubicada esa dirección?")
    
    print(f"\nUsuario: {pais}")
    
    direccion_completa = f"{direccion}, {ciudad}, {pais}"
    print(f"\nMCP: Voy a buscar información para la dirección: {direccion_completa}. ¿Es correcta esta dirección?")
    
    print("\nUsuario: Sí")
    
    print("\nMCP: Perfecto. Estoy localizando la dirección...")
    
    # Buscar resultado mock que coincida con la dirección
    resultado_mock = None
    for resultado in mock_data['mock_resultados']:
        if direccion.lower() in resultado['direccion']['original'].lower():
            resultado_mock = resultado
            break
    
    if not resultado_mock:
        resultado_mock = mock_data['mock_resultados'][0]  # Usar el primero por defecto
    
    # Mostrar resultado
    print(f"\nMCP: Dirección confirmada: {resultado_mock['direccion']['formateada']}")
    print(f"Coordenadas: {resultado_mock['coordenadas']['lat']}, {resultado_mock['coordenadas']['lng']}")
    print(f"Área construida: {resultado_mock['predio']['area_construida']} m²")
    print(f"Estrato: {resultado_mock['predio']['estrato']}")
    print(f"Número predial: {resultado_mock['predio']['numero_predial']}")
    print(f"Uso: {resultado_mock['predio']['uso_descripcion']}")
    
    print("\nCerca tienes:")
    for poi in resultado_mock['pois_cercanos']:
        print(f"- {poi['tipo']}: {poi['nombre']} ({poi['distancia']}m)")
    
    print("\n=== Fin del flujo MCP ===")

# Demostrar flujo completo con la primera dirección de ejemplo
ejemplo = mock_data['demo_direcciones'][0]
simular_flujo_mcp(ejemplo['direccion'], ejemplo['ciudad'], ejemplo['pais'])

## 6. Uso del API REST

Demostración de cómo utilizar el API REST para consultas catastrales.

In [ ]:
# Función para simular llamadas a la API REST
def simular_api_rest(direccion, ciudad, pais):
    print("=== Demostración de uso de la API REST ===")
    
    # Endpoint de geocodificación
    print("\n1. Geocodificación de dirección")
    print("POST /api/catastro/geocode")
    payload = {
        "direccion": direccion,
        "ciudad": ciudad,
        "pais": pais
    }
    print(f"Payload: {json.dumps(payload, indent=2)}")
    
    # Simular respuesta
    for resultado in mock_data['mock_resultados']:
        if direccion.lower() in resultado['direccion']['original'].lower():
            respuesta = {
                "success": True,
                "coordinates": resultado['coordenadas'],
                "formatted_address": resultado['direccion']['formateada']
            }
            break
    else:
        respuesta = {
            "success": True,
            "coordinates": mock_data['mock_resultados'][0]['coordenadas'],
            "formatted_address": mock_data['mock_resultados'][0]['direccion']['formateada']
        }
    
    print(f"Respuesta: {json.dumps(respuesta, indent=2)}")
    
    # Endpoint de consulta completa
    print("\n2. Consulta completa de información catastral")
    print("POST /api/catastro/consulta/completa")
    print(f"Payload: {json.dumps(payload, indent=2)}")
    
    # Simular respuesta
    for resultado in mock_data['mock_resultados']:
        if direccion.lower() in resultado['direccion']['original'].lower():
            respuesta = {
                "success": True,
                "direccion": resultado['direccion'],
                "coordenadas": resultado['coordenadas'],
                "predio": resultado['predio'],
                "pois_cercanos": resultado['pois_cercanos']
            }
            break
    else:
        respuesta = {
            "success": True,
            "direccion": mock_data['mock_resultados'][0]['direccion'],
            "coordenadas": mock_data['mock_resultados'][0]['coordenadas'],
            "predio": mock_data['mock_resultados'][0]['predio'],
            "pois_cercanos": mock_data['mock_resultados'][0]['pois_cercanos']
        }
    
    print(f"Respuesta: {json.dumps(respuesta, indent=2)}")

# Demostrar uso de la API REST con la segunda dirección de ejemplo
ejemplo = mock_data['demo_direcciones'][1]
simular_api_rest(ejemplo['direccion'], ejemplo['ciudad'], ejemplo['pais'])

## 7. Extensibilidad del MCP

Demostración de cómo extender el MCP para incluir nuevas funcionalidades o datasets.

In [ ]:
# Ejemplo de extensión para incluir datos de valor comercial estimado
def estimar_valor_comercial(predio):
    """Estima el valor comercial de un predio basado en sus características."""
    # En un escenario real, esto podría usar un modelo de ML entrenado con datos históricos
    valor_catastral = predio.get('valor_catastral', 0)
    estrato = predio.get('estrato', 3)
    area_construida = predio.get('area_construida', 0)
    
    # Factores de ajuste ficticios para demostración
    factores_estrato = {1: 1.2, 2: 1.3, 3: 1.5, 4: 1.8, 5: 2.2, 6: 2.5}
    factor_estrato = factores_estrato.get(estrato, 1.5)
    
    # Valor comercial estimado
    valor_comercial = valor_catastral * factor_estrato
    
    # Valor por m²
    valor_m2 = valor_comercial / area_construida if area_construida > 0 else 0
    
    return {
        "valor_comercial": valor_comercial,
        "valor_m2": valor_m2,
        "factor_estrato": factor_estrato
    }

# Demostrar extensión con un predio de ejemplo
predio_ejemplo = mock_data['mock_resultados'][2]['predio']
estimacion = estimar_valor_comercial(predio_ejemplo)

print("=== Extensión: Estimación de Valor Comercial ===")
print(f"Predio: {predio_ejemplo['direccion']} ({predio_ejemplo['barrio']})")
print(f"Valor catastral: ${predio_ejemplo['valor_catastral']:,.2f} COP")
print(f"Estrato: {predio_ejemplo['estrato']}")
print(f"Área construida: {predio_ejemplo['area_construida']} m²")
print("\nEstimación:")
print(f"Factor por estrato: {estimacion['factor_estrato']}")
print(f"Valor comercial estimado: ${estimacion['valor_comercial']:,.2f} COP")
print(f"Valor por m²: ${estimacion['valor_m2']:,.2f} COP/m²")

## 8. Conclusiones

Este notebook ha demostrado las principales funcionalidades del MCP de Catastro Geográfico Inteligente:

1. Geocodificación de direcciones con Google Maps API y Nominatim como fallback
2. Consulta de información catastral a partir de direcciones o coordenadas
3. Búsqueda de puntos de interés cercanos a un predio
4. Flujo conversacional completo del MCP
5. Uso de la API REST para integración con otros sistemas
6. Extensibilidad para incluir nuevas funcionalidades

El MCP está diseñado para ser fácilmente integrable en asistentes conversacionales, bots y plataformas digitales, y puede extenderse para incluir nuevos datasets y funcionalidades según las necesidades específicas de cada implementación.